This notebook is thought to be executed in Google colab, but ignoring this first cell can be executed in any jupyter environment.

In [3]:
!pip install -U -q PyDrive
!apt-get install -y libhdf5-serial-dev
!apt-get install -y build-essential swig
!pip install tables
!pip install deap update_checker tqdm stopit
!pip install xgboost
!pip install tpot
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn

!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip install autokeras
!pip install xtoy

!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials. get_application_default()
drive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass


file_list = drive.ListFile(
    {'q': "'parent_key' in parents"}).GetList()

for f in file_list:
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libhdf5-serial-dev is already the newest version (1.10.0-patch1+docs-4).
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   217  100   217    0     0   1382      0 --:--:-- --:--:-- --:--:--  1382
Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
title: oo_data_region.hdf, id: 17FXh20y_J2TOyntDT4BezRXP3ZmDtKVR
downloading to /root/data/oo_data_region.hdf
title: oo_data.hdf, id: 1tHKK1UmvATNakHO43Ius3r7xP_w7I-bx
downloading to /

In [6]:
import pandas as pd
import seaborn as sns
sns.set(style="ticks")
sns.set_palette("husl")
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tpot import TPOTClassifier
import autosklearn.classification
warnings.filterwarnings('ignore')
%matplotlib inline
import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
from xtoy.toys import Toy

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_191"; OpenJDK Runtime Environment (build 1.8.0_191-8u191-b12-0ubuntu0.18.04.1-b12); OpenJDK 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpmqlgpdkp
  JVM stdout: /tmp/tmpmqlgpdkp/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpmqlgpdkp/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_unknownUser_6m2p16
H2O cluster total nodes:,1
H2O cluster free memory:,2.667 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


Loading some helper functions and data

In [0]:
oo_df = pd.read_hdf('/root/data/oo_data_region.hdf', 'data')

oo_df['increase'] = np.sign(oo_df['price_perc'])
oo_jaen_df = oo_df.loc[oo_df['provincia'] == 'JAEN', [ 
        'price_perc-2w', 'price_perc-4w', 'price_perc-6w', 'price_perc-8w',
        'production_tons-1m_sum', 'production_tons-12m_sum', 'production_tons-3m_sum','production_tons-6m_sum', 'production_tons-9m_sum',
        'tmin', 'tmax', 'tmed', 'prec', 'racha', 'prec-4w_sum', 'prec-13w_sum', 'prec-26w_sum', 'prec-52w_sum', 'tmed-4w_mean', 'freeze_damage', 
        'week_nr', 'increase']]
X_train, X_test, y_train, y_test = train_test_split(oo_jaen_df.drop(columns='increase'), oo_jaen_df['increase'], 
                                                    train_size=0.75, test_size=0.25)

In [0]:
tpot_model = TPOTClassifier(generations=50, population_size=100, verbosity=2, n_jobs=-1)
tpot_model.fit(X_train, y_train)
print('TPOT accuracy', tpot_model.score(X_test, y_test))

Generation 1 - Current best internal CV score: 0.5786461608566873


Generation 2 - Current best internal CV score: 0.5918058783321941


Generation 3 - Current best internal CV score: 0.5967596263385737


Generation 4 - Current best internal CV score: 0.6048279790385054


Generation 5 - Current best internal CV score: 0.6048279790385054


...

Generation 47 - Current best internal CV score: 0.6499831396673502


Generation 48 - Current best internal CV score: 0.6499831396673502


Generation 49 - Current best internal CV score: 0.6499831396673502


Generation 50 - Current best internal CV score: 0.6499831396673502



Best pipeline: XGBClassifier(XGBClassifier(MinMaxScaler(PCA(XGBClassifier(input_matrix, learning_rate=0.5, max_depth=1, min_child_weight=2, n_estimators=100, nthread=1, subsample=0.8500000000000001), iterated_power=3, svd_solver=randomized)), learning_rate=0.001, max_depth=8, min_child_weight=12, n_estimators=100, nthread=1, subsample=0.8500000000000001), learning_rate=0.5, max_depth=1, min_child_weight=4, n_estimators=100, nthread=1, subsample=1.0)
TPOT accuracy 0.5511811023622047


In [0]:
autosklearn_model = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=60*60*2)
autosklearn_model.fit(X_train, y_train)
print('Autosklearn accuracy', autosklearn_model.score(X_test, y_test))

[WARNING] [2018-10-22 17:58:00,754:EnsembleBuilder(1):594db4f6123fb2fd68f4eaeae84e2b8c] No models better than random - using Dummy Classifier!

[WARNING] [2018-10-22 17:58:00,770:EnsembleBuilder(1):594db4f6123fb2fd68f4eaeae84e2b8c] No models better than random - using Dummy Classifier![WARNING] [2018-10-22 18:00:12,724:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-10-22 18:00:12,724:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-10-22 18:05:29,909:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-10-22 18:05:29,909:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-10-22 18:05:49,095:smac.intensification.intensification.Intensifier] Challenge

In [9]:
toy = Toy()
toy.fit(X_train, y_train)
print('toy accuracy', toy.score(X_test, y_test))

make_scorer(f1_weighted_score)
unique_combinations 16
Types [2] and maxint [15] detected
--- Evolve in 16 possible combinations ---
gen	nevals	avg     	min     	max     
0  	5     	0.517746	0.503081	0.535273
1  	5     	0.471316	0.42315 	0.510025
2  	2     	0.503759	0.48583 	0.522096
3  	2     	0.500469	0.475086	0.522096
4  	2     	0.508475	0.502639	0.510025
5  	4     	0.519295	0.507935	0.544291
6  	3     	0.502794	0.45437 	0.544291
7  	2     	0.524232	0.481534	0.544291
8  	1     	0.529924	0.472456	0.544291
9  	2     	0.522866	0.463317	0.544291
10 	4     	0.500694	0.466807	0.544291
(0.54429080890645,)
cbp {'clf__alpha': 1e-05}
Best individual is: {'clf__alpha': 1e-05}
with fitness: 0.54429080890645
unique_combinations 216
Types [1, 1, 1, 1, 1, 1] and maxint [5, 3, 2, 2, 0, 0] detected
--- Evolve in 216 possible combinations ---
gen	nevals	avg     	min     	max    
0  	5     	0.539818	0.508885	0.57367
1  	0     	0.56258 	0.54304 	0.57367
2  	3     	0.550368	0.516467	0.57367
3  	1     	0.

In [0]:
train_full_df = X_train.copy()
train_full_df['increase'] = y_train
test_full_df = X_test.copy()
test_full_df['increase'] = y_test
train = h2o.H2OFrame(train_full_df)
test = h2o.H2OFrame(test_full_df)
x = train.columns
y = "increase"
x.remove(y)

train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

h2o_model = H2OAutoML(max_runtime_secs = 60*60*2)
h2o_model.train(x = x, y = y, training_frame = train)
predictions = h2o_model.leader.predict(test).as_data_frame()['predict']
print(h2o_model.leaderboard)
print(h2o_model.leader)
print('H2O accuracy', accuracy_score(predictions, y_test))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%


model_id,mean_per_class_error,logloss,rmse,mse
DeepLearning_grid_0_AutoML_20181022_200359_model_20,0.478262,1.4359,0.603373,0.364059
DeepLearning_grid_0_AutoML_20181022_200359_model_9,0.485635,1.14624,0.589188,0.347143
DeepLearning_grid_0_AutoML_20181022_200359_model_14,0.487166,1.11336,0.592127,0.350615
DeepLearning_grid_0_AutoML_20181022_200359_model_13,0.522563,1.48642,0.635253,0.403546
DeepLearning_grid_0_AutoML_20181022_200359_model_2,0.601024,0.773269,0.518256,0.26859
DeepLearning_grid_0_AutoML_20181022_200359_model_6,0.608398,0.866237,0.534883,0.2861
GBM_grid_0_AutoML_20181022_200359_model_7,0.615021,1.08165,0.66094,0.436842
GBM_grid_0_AutoML_20181022_200359_model_40,0.619407,1.58913,0.604241,0.365107
GBM_grid_0_AutoML_20181022_200359_model_10,0.619407,0.802005,0.535262,0.286506
DeepLearning_grid_0_AutoML_20181022_200359_model_22,0.620805,0.82109,0.537648,0.289065



Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid_0_AutoML_20181022_200359_model_20


ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.013210817787674065
RMSE: 0.11493832166720577
LogLoss: 0.05648928096215949
Mean Per-Class Error: 0.011009066289885787
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



-1,0,1,Error,Rate
148.0,0.0,4.0,0.0263158,4 / 152
0.0,2.0,0.0,0.0,0 / 2
1.0,0.0,148.0,0.0067114,1 / 149
149.0,2.0,152.0,0.0165017,5 / 303


Top-3 Hit Ratios: 


...

Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_classification_error,validation_rmse,validation_logloss,validation_r2,validation_classification_error
,2018-10-22 20:58:27,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan,nan,nan
,2018-10-22 20:58:27,51 min 32.763 sec,16833 obs/sec,10.0,1,3030.0,0.5195775,0.9455903,0.7282187,0.3564356,0.5927612,1.2228237,0.6471464,0.4935065
,2018-10-22 20:58:32,51 min 37.867 sec,24685 obs/sec,430.0,43,130290.0,0.3030178,0.4326937,0.9075610,0.1089109,0.6131536,3.0321350,0.6224508,0.4415584
,2018-10-22 20:58:37,51 min 42.903 sec,26749 obs/sec,910.0,91,275730.0,0.1971509,0.2394587,0.9608694,0.0429043,0.6457557,3.5669843,0.5812338,0.4805195
,2018-10-22 20:58:42,51 min 47.987 sec,28358 obs/sec,1440.0,144,436320.0,0.1233493,0.0477509,0.9846823,0.0231023,0.6860308,4.0355730,0.5273688,0.5064935
,2018-10-22 20:58:47,51 min 53.005 sec,29560 obs/sec,1990.0,199,602970.0,0.1468852,0.0726813,0.9782793,0.0297030,0.6701449,4.2744237,0.5490042,0.5064935
,2018-10-22 20:58:52,51 min 58.014 sec,30537 obs/sec,2560.0,256,775680.0,0.1178255,0.0402937,0.9860235,0.0198020,0.6695263,4.5886882,0.5498363,0.5194805
,2018-10-22 20:58:55,52 min 0.918 sec,31049 obs/sec,2900.0,290,878700.0,0.1149383,0.0564893,0.9867001,0.0165017,0.6611118,4.5784379,0.5610805,0.4675325


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
price_perc-2w,1.0,1.0,0.0834804
prec,0.8934146,0.8934146,0.0745826
prec-4w_sum,0.7625359,0.7625359,0.0636568
freeze_damage,0.7529436,0.7529436,0.0628560
racha,0.6696827,0.6696827,0.0559054
---,---,---,---
week_nr,0.4440292,0.4440292,0.0370677
tmin,0.4221320,0.4221320,0.0352397
tmax,0.4131543,0.4131543,0.0344903
tmed-4w_mean,0.3671600,0.3671600,0.0306507



See the whole table with table.as_data_frame()

H2O accuracy 0.5118110236220472


In [0]:
h2o_model.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
GBM_grid_0_AutoML_20181021_095225_model_11,0.598233,0.93755,0.606634,0.368004
GBM_grid_0_AutoML_20181021_095225_model_5,0.606843,1.0274,0.641612,0.411666
GBM_grid_0_AutoML_20181021_095225_model_6,0.623138,1.03253,0.643644,0.414277
GBM_grid_0_AutoML_20181021_095225_model_2,0.631958,0.858713,0.551327,0.303962
GBM_grid_0_AutoML_20181021_095225_model_1,0.636942,0.874619,0.556777,0.31
DeepLearning_0_AutoML_20181021_095225,0.639836,0.800127,0.527211,0.277951
GBM_grid_0_AutoML_20181021_095225_model_10,0.641717,0.786424,0.531643,0.282645
GBM_grid_0_AutoML_20181021_095225_model_0,0.643164,0.945412,0.575628,0.331347
GBM_grid_0_AutoML_20181021_095225_model_9,0.646477,1.03677,0.645291,0.416401
GLM_grid_0_AutoML_20181021_095225_model_0,0.647014,0.756689,0.515234,0.265466
